|Anchors                |Description                                |
|-----------------------|-------------------------------------------|
| \A                    | match start of string                     |
| \Z                    | match end of string                       |
| ^                     | match start of line                       |
| $                     | end of line                               |
| \b                    | start/end of words                        |
| \B                    | inverse of \b                             |

|Groups| Description|
|------------------------|------------------------------------------|
| (?:)                  | non-capturing                             |
| (?P<name>)            | named capture group                       |
| .                     | any except \n                             |
| []                    | character classes                         |
| *                     | 0 or more                                 |
| +                     | 1 or more                                 |
| ?                     | 0 or 1                                    |
| {3,5}                 | 3 to 5 times                              |
| pat1.*pat2|pat2.*pat1 | match both pat1 and pat2 but in any order |

| Character classes | Description                                         |
|-------------------|-----------------------------------------------------|
| [ABC]             | Match any character in the set                      |
| [^ABC]            | Match any character not in the set                  |
| [A-z]             | Matches a range                                     |
| .                 | Match any except linebreaks. Shortcut for [^\n\r]   |
| \w                | Match word chars. Shortcut for [A-Za-z0-9_]         |
| \W                | Negated ^w. Shortcut for [^A-Za-z0-9_]              |
| \d                | Shortcut for [0-9]                                  |
| \D                | Shortcut for [^0-9]                                 |
| \s                | Whitespace                                          |
| [\uxxx-\uxxy]     | Match a character in range (see below)               |

In [6]:
s = "ῷ"
s.encode("unicode_escape")


b'\\u1ff7'